#TopEditor - traitements NER et enrichissements des textes TEI

Le dossier du projet se trouve ici :
```
/content/drive/MyDrive/Colab Notebooks/TopEditor/
```
Description des sous-dossiers associés aux traitements:
* **NERmodels** : les modèles NER produits de reperage d'entités nommées (et autres)

* **data/TEI/Final/input**: fichiers TEI stylé via Word customisé Metopes et converti en schéma métopes (via XMLMind)

* **data/TEI/Final/clean**: fichiers TEI du dossier input legerement "nettoyés" manuellement du bruit produit par Word après consignes Métopes

* **data/TEI/Final/tagged**: fichiers TEI avec balisage automatisé via un modèle NER custom

* **data/TEI/Final/reference_pers**: fichiers tableurs avec les identifiants personnes à désambiguiser manuellement

* **data/TEI/Final/reference_place**: fichiers tableurs avec les identifiants place (sens restreint) à désambiguiser manuellement

* **data/TEI/Final/tagged/updated_person**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes

* **data/TEI/Final/tagged/updated_person_upated_place**: fichiers dans "tagged" avec les identifiants (attribut ref) pour les personnes et pour les lieux

* **data/TEI/Final/tagged_corrected_Sara**: fichiers TEI dans updated_person_upated_place avec la correction manuelle par Sara via Oxygène des balises fautives

* **data/TEI/Final/tagged_corrected_Sara/TagAugmented** fichiers dans tagged_corrected_Sara avec un balisage automatisé par reglès des dates déjà reperées par le modèle NER et pas du tout repérées




In [1]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount('/content/drive')

Mounted at /content/drive


## Corrections et ajout de balises date dans des fichiers TEI corrigés par Sara



### **1e passe**: ajout de balises pour des dates en format complet et suppression d'erreurs des dates érronément balisées

In [2]:
import os
import re
import shutil
import lxml.etree as ET
from pathlib import Path

DATE_REGEX = r'\b(?<!\d)(\d{1,2}/\d{1,2}/\d{4})(?!\d)\b'  # Ex: "12/09/1425"

def detect_dates(text):
    """Détecte les dates dans un texte en excluant celles déjà balisées."""
    date_patterns = [
        (r'(?<!\d)(\d{1,2}/\d{1,2}/\d{4})(?!\d)', 'full')  # Format jj/mm/aaaa
    ]
    matches = []
    for pattern, date_type in date_patterns:
        for match in re.finditer(pattern, text):
            matches.append((match.start(), match.end(), match.group(), date_type))
    return matches

def wrap_dates_in_tei(paragraph):
    """
    Ajoute des balises <date> autour des dates non encore balisées,
    sans supprimer les autres éléments de la structure XML.
    """
    existing_dates = {el.text for el in paragraph.findall('.//{*}date') if el.text}

    # Création d'une liste pour stocker les nouveaux enfants du paragraphe
    new_children = []

    # Parcours des enfants existants
    for elem in paragraph:
        new_children.append(elem)  # Ajouter l'élément inchangé à la nouvelle liste

    # Traitement du texte du paragraphe
    text = paragraph.text
    if text:
        new_paragraph_content = []
        last_index = 0

        for match in re.finditer(DATE_REGEX, text):
            date_text = match.group()
            if date_text in existing_dates:
                continue  # Ne pas dupliquer les dates déjà balisées

            # Ajouter le texte avant la date
            if last_index < match.start():
                new_paragraph_content.append(text[last_index:match.start()])

            # Ajouter un élément <date> en tant que nœud XML
            date_element = ET.Element("date")
            date_element.text = date_text
            new_paragraph_content.append(date_element)

            last_index = match.end()

        # Ajouter le texte restant après la dernière date
        if last_index < len(text):
            new_paragraph_content.append(text[last_index:])

        # Réécrire le contenu du paragraphe
        paragraph.text = ""
        for part in new_paragraph_content:
            if isinstance(part, ET._Element):
                paragraph.append(part)  # Ajouter le nœud XML <date>
            else:
                if paragraph.text:
                    paragraph[-1].tail = part  # Ajouter le texte après une balise existante
                else:
                    paragraph.text = part  # Premier élément texte du paragraphe

    # Réinsérer les éléments enfants d'origine après modification du texte
    paragraph.extend(new_children)

def update_date_attributes(output_dir):
    """Relit les fichiers XML avec des balises <date>, remplit les attributs when et supprime les attributs to/from restants."""
    output_path = Path(output_dir)
    for file in output_path.glob("*.xml"):
        parser = ET.XMLParser(remove_blank_text=True)
        tree = ET.parse(file, parser)
        root = tree.getroot()

        for paragraph in root.findall(".//{*}p[@rend='TEI_localillDOI']"):
            for date_elem in paragraph.findall('.//{*}date'):
                # Remplacez l'élément <date> par son contenu textuel
                if date_elem.tail:
                    # Si l'élément <date> a un 'tail', il faut le conserver
                    parent = date_elem.getparent()
                    previous = date_elem.getprevious()
                    if previous is not None:
                        previous.tail = (previous.tail or '') + date_elem.text + date_elem.tail
                    else:
                        parent.text = (parent.text or '') + date_elem.text + date_elem.tail
                    parent.remove(date_elem)
                else:
                    date_elem.getparent().text = (date_elem.getparent().text or '') + date_elem.text
                    date_elem.getparent().remove(date_elem)

        for paragraph in root.findall(".//{*}p"):
            for date_elem in paragraph.findall('.//{*}date'):
                # Remplacez l'élément <date> par son contenu textuel
                date_text = (date_elem.text or '').strip()

                # Cas où la date est un nombre à 3 chiffres (ex: <date>935</date>)
                if re.match(r'\b\d{3}\b', date_text) or re.match(r'\(|\)', date_text):
                    print(f"Suppression de la balise <date> avec contenu: {date_text}")

                    if date_elem.tail:
                        # Si l'élément <date> a un 'tail', il faut le conserver
                        parent = date_elem.getparent()
                        previous = date_elem.getprevious()
                        if previous is not None:
                            previous.tail = (previous.tail or '') + date_elem.text + date_elem.tail
                        else:
                            parent.text = (parent.text or '') + date_elem.text + date_elem.tail
                        parent.remove(date_elem)
                    else:
                        date_elem.getparent().text = (date_elem.getparent().text or '') + date_elem.text
                        date_elem.getparent().remove(date_elem)

        for date_elem in root.findall('.//{*}date'):
            date_text = (date_elem.text or '').strip()
            if 'from' in date_elem.attrib:
                del date_elem.attrib['from']
            if 'to' in date_elem.attrib:
                del date_elem.attrib['to']
            if re.match(r'\d{1,2}/\d{1,2}/\d{4}', date_text):
                date_elem.set('when', date_text)

        tree.write(file, encoding='utf-8', pretty_print=True, xml_declaration=True)

def process_file(input_path, output_path):
    """Traite un fichier XML/TEI en ajoutant des balises <date> autour des expressions de dates détectées."""
    parser = ET.XMLParser(remove_blank_text=True)
    tree = ET.parse(input_path, parser)
    root = tree.getroot()
    namespaces = root.nsmap  # Récupération des namespaces

    for paragraph in root.findall(".//{*}p[@rend='TEI_localparaDate']") + root.findall(".//{*}p[@rend='TEI_localparaDonnees']"):
        wrap_dates_in_tei(paragraph)

    tree.write(output_path, encoding='utf-8', pretty_print=True, xml_declaration=True)

def process_directory(input_dir, output_dir):
    """Traite tous les fichiers XML/TEI d'un dossier donné."""
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    for file in input_path.glob("*.xml"):
        print(file)
        process_file(file, output_path / file.name.replace(".xml", "_enrichCarmen.xml"))

    update_date_attributes(output_dir)

if __name__ == "__main__":
    ## 1e passe: reperage dates format complet
    dossier_entree = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara"
    dossier_sortie = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented"

    process_directory(dossier_entree, dossier_sortie)
    print(f"Traitement terminé. Les fichiers modifiés sont dans {dossier_sortie}")



/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara.xml
Suppression de la balise <date> avec contenu: 941
Suppression de la balise <date> avec contenu: 959
Suppression de la balise <date> avec contenu: 939
Suppression de la balise <date> avec contenu: 940
Suppression de la balise <date> avec contenu: 955
Suppression de la balise <date> avec contenu: 966
Suppression de la balise <date> avec contenu: 929
Suppression de la balise <date> avec contenu: 955
Suppression de la balise <date> avec contenu: 960
Suppression de la balise <date> avec contenu: 963
Suppression de la balise <date> avec contenu: 929
Suppression de la balise <date> avec contenu: 935
Suppression de la balise <date> avec contenu: 936
Suppression de la balise <date> avec contenu: 941
Suppression de la balise <date> avec contenu: 957
Suppression de la balise <date> avec contenu: 963
Suppression de la balise <date

### **2e passe**: ajout de balises des dates en format année (car conflits de superposition des balises en une seule passe)

In [3]:
import os
import re
import shutil
import lxml.etree as ET
from pathlib import Path

DATE_REGEX = r'\b(?<!\d)(\d{4})(?!\d)\b'  # Ex: "1425"

def detect_dates(text):
    """Détecte les dates dans un texte en excluant celles déjà balisées."""
    date_patterns = [
        (r'(?<!\d)(\d{4})(?!\d)', '{year}') # Format aaaa
    ]
    matches = []
    for pattern, date_type in date_patterns:
        for match in re.finditer(pattern, text):
            matches.append((match.start(), match.end(), match.group(), date_type))
    return matches

def wrap_dates_in_tei(paragraph):
    """
    Ajoute des balises <date> autour des dates non encore balisées,
    sans supprimer les autres éléments de la structure XML.
    """
    existing_dates = {el.text for el in paragraph.findall('.//{*}date') if el.text}

    # Création d'une liste pour stocker les nouveaux enfants du paragraphe
    new_children = []

    # Parcours des enfants existants
    for elem in paragraph:
        new_children.append(elem)  # Ajouter l'élément inchangé à la nouvelle liste

    # Traitement du texte du paragraphe
    text = paragraph.text
    if text:
        new_paragraph_content = []
        last_index = 0

        for match in re.finditer(DATE_REGEX, text):
            date_text = match.group()
            if date_text in existing_dates:
                continue  # Ne pas dupliquer les dates déjà balisées

            # Ajouter le texte avant la date
            if last_index < match.start():
                new_paragraph_content.append(text[last_index:match.start()])

            # Ajouter un élément <date> en tant que nœud XML
            date_element = ET.Element("date")
            date_element.text = date_text
            new_paragraph_content.append(date_element)

            last_index = match.end()

        # Ajouter le texte restant après la dernière date
        if last_index < len(text):
            new_paragraph_content.append(text[last_index:])

        # Réécrire le contenu du paragraphe
        paragraph.text = ""
        for part in new_paragraph_content:
            if isinstance(part, ET._Element):
                paragraph.append(part)  # Ajouter le nœud XML <date>
            else:
                if paragraph.text:
                    paragraph[-1].tail = part  # Ajouter le texte après une balise existante
                else:
                    paragraph.text = part  # Premier élément texte du paragraphe

    # Réinsérer les éléments enfants d'origine après modification du texte
    paragraph.extend(new_children)

def update_date_attributes(output_dir):
    """Relit les fichiers XML avec des balises <date>, remplit les attributs when et supprime les attributs to/from restants."""
    output_path = Path(output_dir)
    for file in output_path.glob("*.xml"):
        parser = ET.XMLParser(remove_blank_text=True)
        tree = ET.parse(file, parser)
        root = tree.getroot()

        for date_elem in root.findall('.//{*}date'):
            date_text = (date_elem.text or '').strip()
            if re.match(r'\d{4}', date_text):
                date_elem.set('when', date_text)

        tree.write(file, encoding='utf-8', pretty_print=True, xml_declaration=True)

def process_file(input_path, output_path):
    """Traite un fichier XML/TEI en ajoutant des balises <date> autour des expressions de dates détectées."""
    parser = ET.XMLParser(remove_blank_text=True)
    tree = ET.parse(input_path, parser)
    root = tree.getroot()
    namespaces = root.nsmap  # Récupération des namespaces

    for paragraph in root.findall(".//{*}p[@rend='TEI_localparaDate']") + root.findall(".//{*}p[@rend='TEI_localparaDonnees']"):
        wrap_dates_in_tei(paragraph)

    tree.write(output_path, encoding='utf-8', pretty_print=True, xml_declaration=True)

def process_directory(input_dir, output_dir):
    """Traite tous les fichiers XML/TEI d'un dossier donné."""
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    for file in input_path.glob("*.xml"):
        print(file)
        process_file(file, output_path / file.name)

    update_date_attributes(output_dir)

if __name__ == "__main__":
    ## 1e passe: reperage dates format complet
    dossier_entree = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented"
    dossier_sortie = "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented"

    process_directory(dossier_entree, dossier_sortie)
    print(f"Traitement terminé. Les fichiers modifiés sont dans {dossier_sortie}")



/content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented/semtags_2combiné_T2 clj. San Pedro_clean_updated_person_updated_place_corrSara_enrichCarmen.xml
Traitement terminé. Les fichiers modifiés sont dans /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/Final/tagged_corrected_Sara/TagAugmented


**Problèmes restants à corriger cas par cas**:
- des dates mal ortographiées qui sont souvent reperées par le NER mais mal interpretées en post-traitement
```
<date>5/101486</date>
```
- Il y en revanche des expressions bien reperées par le NER comme <date to="x" from="x">Toussaint 1393</date>, il faudrait manuellement remplir les attributs pour ce cas spécifique.


**Problèmes corrigés automatiquement en post-traitement**:
- fausses dates à trois chiffres, il y a des erreurs attendues, par exemple de faux positifs, cet exemple vient du NER :
```
f° 37 r°. OF <date to="x" from="x">935</date>, f° 44 r°..</p>
```
- autre problème de l'injection du NER :
```
<p rend="TEI_localillDOI" xml:id="p156">/nkl.93dc8nr4/8e8ee596b73b15edcd8205c1f77cf2296729cc9c<date to="x" from="x">10.34847</date></p>
```
Ce problème a été resolu en post-traitement, il faut sélectionner des paragraphes (deux valeurs possibles attribut "rend" pour p), la fonction d'injection NER a été mise à pour le traitement des nouveaux fichiers.
- parenthèses balisées comme dates (très étrange) :
```
<date>(</date><date>1408</date>
```




Ver notebook de "control"